In [1]:
%matplotlib inline

import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

import QSTK.qstkutil.qsdateutil as du
import QSTK.qstkutil.tsutil as tsu
import QSTK.qstkutil.DataAccess as da
import QSTK.qstkstudy.EventProfiler as ep

In [30]:
def find_events(symbols, data):
    close = data['close']
    market = close['SPY']
    events = pd.DataFrame(np.full_like(close, np.NaN), columns=close.columns)
    timestamps = close.index
    
    for symbol in symbols:
        for i in range(1, len(timestamps)):
            symbolprice_today = close[symbol].ix[timestamps[i]]
            symbolprice_yesterday = close[symbol].ix[timestamps[i-1]]
            marketprice_today = market.ix[timestamps[i]]
            marketprice_yesterday = market.ix[timestamps[i-1]]
            
            symbol_return = (symbolprice_today/symbolprice_yesterday) - 1
            market_return = (marketprice_today/marketprice_yesterday) - 1
            
            if symbol_return <= -0.03 and market_return >= 0.02:
                events[symbol].ix[timestamps[i]] = 1
                
    return events

In [31]:
start_date = dt.datetime(2008, 1, 1)
end_date = dt.datetime(2009, 12, 31)
timestamps = du.getNYSEdays(start_date, end_date, dt.timedelta(hours=16))

dataobject = da.DataAccess('Yahoo')
symbols = dataobject.get_symbols_from_list('sp5002012')
symbols.append('SPY')

keys = ['open', 'high', 'low', 'close', 'volume', 'actual_close']
data = dataobject.get_data(timestamps, symbols, keys)
data = dict(zip(keys, data))

for key in keys:
    data[key] = data[key].fillna(method='ffill')
    data[key] = data[key].fillna(method='bfill')
    data[key] = data[key].fillna(1.0)
    
events = find_events(symbols, data)


/Users/chico/venvs/ci/lib/python2.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':
/Users/chico/venvs/ci/lib/python2.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  # This is added back by InteractiveShellApp.init_path()
/Users/chico/venvs/ci/lib/python2.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


In [34]:
events['A' == True]

KeyError: False

In [26]:
ep.eventprofiler(events, data, i_lookback=20, i_lookforward=20, s_filename='MyEventStudy.pdf', s_market_sym='SPY')

AssertionError: Zero events in the event matrix